# Residential Building Classification
This is a bare-bone version of the residential vs. non-residential building classification. We use DenseNet in this implementation. 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision import transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt

import params
from densenet_efficient import DenseNetEfficient
from solver import *
from dataset import *
from visualize import *

# Load parameters
params = vars(params)
rbc_class_names = params['rbc_class_names']
fmow_class_names = params['fmow_class_names']
fmow_class_names_mini = params['fmow_class_names_mini']
dtype = params['dtype']

torch.set_num_threads(params['num_threads'])

## Preprocess FMOW Dataset
Check if fMoW dataset is available. If so, process the images by cropping based on given bounding boxes. Load in traing, validation and test dataset along with ground truth labels for training and validation set. 

In [2]:
# Wake Data Transformation
# train_transforms = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(3/4, 4/3)),
#     transforms.ToTensor(),
# ])
# fMoW Data Transformation
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0), ratio=(4/5, 5/4)),
    transforms.ToTensor(),
])
train_set = FMOWDataset(params, transform=train_transforms)
print("Number of Training/Validation Images: %i" % len(train_set))
x,y = train_set[0]
print(type(x), x.shape, type(y), y.shape)

Number of Training/Validation Images: 11833
<class 'torch.Tensor'> torch.Size([1, 224, 224]) <class 'torch.Tensor'> torch.Size([1])


In [3]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
test_set = FMOWDataset(params, transform=test_transforms, train=False)
print("Number of Testing Images: %i" % len(test_set))
x,y = test_set[0]
print(type(x), x.shape, type(y), y.shape)

Number of Testing Images: 240


KeyError: 'category'

### Visualize a small batch of data
Plot a small batch of training images to get a sense of data

In [4]:
# Sample a minibatch and show the images and captions
size = 5
indices = np.random.choice(len(test_set), size, replace=False)

fig = plt.figure()

for i, idx in enumerate(indices):
    image, label = test_set[idx]
    plt.subplot(1, size, i + 1)
    plt.title(params['rbc_class_names'][label[0]])
    plt.axis('off')
    plt.imshow(image[0,:,:])
    plt.gcf().set_size_inches(10, 10)
    plt.show()

KeyError: 'category'

<Figure size 432x288 with 0 Axes>

## Load Model
Load in DenseNet. Configure training parameters. 

In [3]:
model = DenseNetEfficient(
        growth_rate=32, 
        block_config=(6, 12, 24, 16), 
        compression=0.5,
        num_init_features=64, 
        bn_size=4, 
        drop_rate=0,
        num_classes=20, 
        small_inputs=False)
# Move model to GPU
model = model.cuda()
# print(model)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Parameters: %i" % pytorch_total_params)

Total Parameters: 6968084


/home/yuansong/anaconda2/envs/rbc/lib/python3.6/site-packages/torch/nn/modules/container.py:16: UserWarning: nn.Container is deprecated. All of it's functionality is now implemented in nn.Module. Subclass that instead.
  warnings.warn("nn.Container is deprecated. All of it's functionality "


In [ ]:
# Modify the model to predict 2 categories
# model.classifier = nn.Sequential(
#               nn.Linear(1024, 128),
#               nn.ReLU(),
#               nn.Linear(128, 2))

In [12]:
# Load an existing model trained on 20 categories
# model.load_state_dict(torch.load('./models/cat_2_fmow_50000_pure_wake_12000/model.dat'))

RuntimeError: Error(s) in loading state_dict for DenseNetEfficient:
	Missing key(s) in state_dict: "features.denseblock1.denselayer1.norm2.weight", "features.denseblock1.denselayer1.norm2.bias", "features.denseblock1.denselayer1.norm2.running_mean", "features.denseblock1.denselayer1.norm2.running_var", "features.denseblock1.denselayer1.conv2.weight", "features.denseblock1.denselayer2.norm2.weight", "features.denseblock1.denselayer2.norm2.bias", "features.denseblock1.denselayer2.norm2.running_mean", "features.denseblock1.denselayer2.norm2.running_var", "features.denseblock1.denselayer2.conv2.weight", "features.denseblock1.denselayer3.norm2.weight", "features.denseblock1.denselayer3.norm2.bias", "features.denseblock1.denselayer3.norm2.running_mean", "features.denseblock1.denselayer3.norm2.running_var", "features.denseblock1.denselayer3.conv2.weight", "features.denseblock1.denselayer4.norm2.weight", "features.denseblock1.denselayer4.norm2.bias", "features.denseblock1.denselayer4.norm2.run

## Train Model
Train the model one epoch at a time. Keep the verbose option on to monitor the loss during training. 
Save training and validation accuracy history for visualization. 

In [4]:
train(model, 
      train_set, 
      None, 
      './output', 
      n_epochs=30, 
      valid_size=1000,
      batch_size=50, 
      lr=0.001, 
      wd=0.0001, 
      momentum=0.9, 
      seed=None)

torch.LongTensor


TypeError: unhashable type: 'numpy.ndarray'

## Loss History
Visualize training and validation accuracy throughout training history. 
Visualize loss function. 

In [ ]:
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

with open(os.path.join('./output', 'results.csv'), 'r') as f:
        for line in f:
            numbers = line.split(',')
            if(numbers[0]=='epoch'):
                continue
            try:
                train_loss.append(float(numbers[1]))
                train_acc.append(1-float(numbers[2]))
                valid_loss.append(float(numbers[3]))
                valid_acc.append(1-float(numbers[4]))
            except ValueError:
                continue


# Visualize loss history and training/validation accuracy history
plt.subplot(2, 1, 1)
plt.plot(train_loss, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(train_acc, '-o')
plt.plot(valid_acc, '-o')
plt.legend(['train', 'val'], loc='lower right')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

## Test Model
Make predictions based on test dataset. Visually assess the quality of prediction and compute accuracy based on groundtruth. 

In [ ]:
# Check accuracy of model on Test set
test_loader = torch.utils.data.DataLoader(test_set, batch_size=50, shuffle=False,
                                              pin_memory=(torch.cuda.is_available()), num_workers=0)
# model.cuda()
# test_epoch(model, test_loader)

## Visualization on Test Images

In [ ]:
np.random.seed(5)
# Load some test images for visualization
X_vis = None
y_vis = None
vis_size = 10
vis_indices = np.random.choice(len(test_set), vis_size, replace=False)
for i, idx in enumerate(vis_indices):
    if X_vis is None:
        X_vis, y_vis = test_set[idx]
    else:
        X_t, y_t = test_set[idx]
        X_vis = torch.cat([X_vis, X_t], 0)
        y_vis = torch.cat([y_vis, y_t], 0)
X_vis = X_vis.type(torch.FloatTensor) 
X_vis = torch.unsqueeze(X_vis, 1)
print(X_vis.size(), type(X_vis), y_vis.size(), type(y_vis))

# Pick out success and failure cases
model.cuda()
model.eval() 
X_vis_var = Variable(X_vis.cuda(async=True))
output = model(X_vis_var)
_, preds = output.data.cpu().max(1)
idx_success = (preds == y_vis).numpy() == 1
idx_failure = (preds != y_vis).numpy() == 1
print(idx_success)
print(idx_failure)
if idx_success.any():
    X_success = torch.from_numpy(X_vis.numpy()[idx_success])
    y_success = torch.from_numpy(y_vis.numpy()[idx_success])
else:
    X_success = torch.FloatTensor([])
    y_success = torch.LongTensor([])
if idx_failure.any():
    X_failure = torch.from_numpy(X_vis.numpy()[idx_failure])
    y_failure = torch.from_numpy(y_vis.numpy()[idx_failure])
else:
    X_failure = torch.FloatTensor([])
    y_failure = torch.LongTensor([])

### Class Activation Map

In [ ]:
vis_success = True

X_vis = X_success if vis_success else X_failure
y_vis = y_success if vis_success else y_failure
cams = getCAM(X_vis, y_vis, model)
vis_size = len(X_vis)

fig = plt.figure()
for i in range(vis_size):
    cam = cams[i]
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    image = X_vis.numpy()[i]
    image = np.moveaxis(image, 0, 2)
    overlay = heatmap * 0.4 + image * 0.5
    overlay = np.uint8(overlay)
    
    plt.subplot(1, vis_size*2, i + 1)
    plt.imshow(overlay)
    plt.axis('off')
    plt.title('Heat Map for Correct Class: %s' % rbc_class_names[y_vis[i]])
    plt.subplot(1, vis_size*2, i + vis_size + 1)
    plt.imshow(image[:,:,0])
    plt.axis('off')
    plt.gcf().set_size_inches(24, 8)
    plt.show()

### Visualize Successful and Failed Examples

In [ ]:
# X_success = np.array([]).reshape((0, 1, 224, 224))
# y_success = np.array([]).reshape((0,))
# X_failure = np.array([]).reshape((0, 1, 224, 224))
# y_failure = np.array([]).reshape((0,))

# model.eval() 
# for X, y in test_loader:
#     X_var = Variable(X.type(dtype), volatile=True)
#     scores = model(X_var)
#     _, preds = scores.data.cpu().max(1)
#     y = y[:,0]
#     idx_success = (preds == y).numpy() == 1
#     idx_failure = (preds != y).numpy() == 1
#     X = X.numpy()
#     y = y.numpy()
#     X_success = np.append(X_success, X[idx_success.tolist(), :, :, :], axis=0)
#     y_success = np.append(y_success, y[idx_success.tolist(), ], axis=0)
#     X_failure = np.append(X_failure, X[idx_failure.tolist(), :, :, :], axis=0)
#     y_failure = np.append(y_failure, preds[idx_failure.tolist(), ], axis=0)
    
# y_success = y_success.astype(np.int32)
# y_failure = y_failure.astype(np.int32)

In [ ]:
# # Sample a batch of successfully classified images
# for i in range(10):
#     plt.subplot(3, 4, i+1)
#     img = np.moveaxis(X_success[i, :, :, :], 0, -1)
#     plt.imshow(img[:,:,0])
#     plt.axis('off')
#     plt.title(rbc_class_names[y_success[i]])
#     plt.show()

In [ ]:
## Sample a batch of incorrectly classified images
# plt.title('Failure Examples')
# for i in range(8):
#     plt.subplot(2, 4, i+1)
#     img = np.moveaxis(X_failure[i, :, :, :], 0, -1)
#     plt.imshow(img[:,:,0])
#     plt.axis('off')
#     plt.title(rbc_class_names[y_failure[i]])
#     plt.show()